# Exercise 1: t-SNE

## Do not start the exercise until you fully understand the submission guidelines.


* The homework assignments are executed automatically. 
* Failure to comply with the following instructions will result in a significant penalty. 
* Appeals regarding your failure to read these instructions will be denied. 
* Kind reminder: the homework assignments contribute 60% of the final grade.


## Read the following instructions carefully:

1. This Jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Write **efficient**, **vectorized** code whenever possible. Some calculations in this exercise may take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deductions.
1. You are responsible for the correctness of your code and should add as many tests as you see fit to this jupyter notebook. Tests will not be graded nor checked.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/).
1. Your code must run without errors. Use at least `numpy` 1.15.4. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Submission includes a zip file that contains this notebook, with your ID as the file name. For example, `hw1_123456789_987654321.zip` if you submitted in pairs and `hw1_123456789.zip` if you submitted the exercise alone. The name of the notebook should follow the same structure.
   
Please use only a **zip** file in your submission.

---
---

## Please sign that you have read and understood the instructions: 

316492776
318900016

---
---


In [ ]:
# Required dependencies
!pip3 install numpy scipy pandas scikit-learn matplotlib mpl-tools tqdm seaborn

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.manifold import TSNE
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    confusion_matrix,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm.auto import tqdm

sns.set_theme()
np.random.seed(42)

# Design your algorithm
Make sure to describe the algorithm, its limitations, and describe use-cases.

tSNE is a non-linear dimensionality reduction algorithm, used mostly for the purpose of visualization in 2d and sometimes 3d.
Its input is $n$ points in a high dimentional space $\mathbb{R}^h$, and its output is $n$ correspoinding points in a $\mathbb{R}^2$ (or $\mathbb{R}^3$).

The algorithm calculates pairwise distances is the original space, tranlates them to a "similarity" probabilty distribution $p_{ij}$ using gaussians, and tries to make the similarities in the lower dimensional space $q_{ij}$ have that same distribution (this time with T-distribution - more about that below).

Its cost function is KL-divergence between P and Q, which represents "distance" between distribution. It is minimized using gradient decsent.

KL-devergence defenition, $\sum_{i,j=1}^{N}{p_{ij} \cdot \log{\frac{p_{ij}}{q_{ij}}}}$, causes higher $p_{ij}$ values to be more important than smaller ones, meaning tSNE generally focuses on preserving local structure (=higher $p_{ij}$ values corresponding to closer points in the original space) but sometimes fail to capture global relationships.

This property, however convinient for visualization, means that tSNE is generally not a good choice to be used as a more "mathematical" option for dimensionality reduction, since it does not preserve basic geometric features of the data in the original space.

Unlike regular SNE, tSNE is symetric, and uses T-distribution in the lower dimentional space in order to avoid shrinkage to a single point or a very dense cluster (since T-distribution has a heavy tail, unlike a gaussian which does not).

The (important) hyperparameters:
- n_components: the low dimension (almost always 2 or 3)
- perplexity: relates to the amount of neighbors to be considered when calculating $\sigma_i$
- n_iter: the maximal number of iterations of the gradient descent process.
- learning rate: learning rate for the gradient descent algorithm. High learning rate leads to faster loss decrease but may also fail to converge at later iterations. Low learning rates suffer less from that, but convergence more slowly and require more steps (and time) to reach small loss values. SKLearn implements "auto" method, which changes lr over time.
- method: SKLearn supports the 'barnes_hut' distance approximations, which causes $q_{ij}$ to be $O(n \log{n})$ rather than $O(n^2)$. It is less precise, but makes the algorithm way more scalable. Exact calculations become impractical with thousands of examples since $O(n^2)$ increases fast, while $O(n \log{n})$ still performs well.

# Your implementations
You may add new cells, write helper functions or test code as you see fit.
Please use the cell below and include a description of your implementation.
Explain code design consideration, algorithmic choices and any other details you think is relevant to understanding your implementation.
Failing to explain your code will lead to point deductions.

We implement t-SNE as seen in the class, more specifically according to slides 61-66.
The calculation of $q_{ij}$ and $p_{ij}$ is pretty straight forward according to the theorey shown in class.

We implemented a search to find $\sigma_i$ for each example, as suggested in the piazza forum. For each example, we choose $\sigma_i$ that achieves the desired perplexity. We implemented this by using a vectorized binary search (for all $\sigma_i$ together), increasing or descreasing sigma to get closer to the desired perplexity. Since we cut the search space by half each iteration, the process converges fast.
The (configurable) default sigma values are between $10^{-3}$ and $10^3$ with precision of $10^{-4}$, which covers more than enough values (when we ran the algorithm, final sigma values were all between $10^{-1}$ and $10^1$)

We did not implement 'barnes_hut' optimization and we stick to the exact method since, meaning we don't scale so well for large numbers of examples. We did, however, make sure to write vectorized code to maximize efficiency and speed as much as possible, so we handle thousands of examples with little effort. 

We also did not implement lr decay methods, since using relatively low values and increasing the max steps seemed to do the job (graphs later), and did not take too much time to run.

The optimization is done by gradient descent process, with an option for early stopping if no progress is made after some time.

We added few more parameters to the original signature:
- n_iter_without_progress: number of iterations to wait before early stopping if the loss hasn't decreased.
- verbose: verbosity, used to help us debug (helpful prints and progress bars)
- min_sigma & max_sigma: control the range of values that $\sigma_i$ can get
- transform_n_neighbors: the number of neighbors from the train to be considered when calculating the transform method.

In [ ]:
class CustomTSNE:
    def __init__(
        self,
        perplexity: float = 30.0,
        n_components: int = 2,
        n_iter: int = 1000,
        learning_rate: float = 200.0,
        n_iter_without_progress: int = 300,
        verbose: bool = False,
        min_sigma: float = 1e-3,
        max_sigma: float = 1e3,
        transform_n_neighbors: int = 10,
    ):
        self.perplexity = perplexity
        self.n_components = n_components
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        # Note: You may add more attributes
        self.n_iter_without_progress = n_iter_without_progress
        self.verbose = verbose
        self.min_sigma = min_sigma
        self.max_sigma = max_sigma
        self.transform_n_neighbors = transform_n_neighbors

    def fit_transform(self, X):
        # Part 1: Implementing t-SNE

        # Step 1: Compute pairwise affinities in the original space with a Gaussian distribution
        # Your code here
        X = X / np.max(X)
        N = len(X)
        X_dists = squareform(pdist(X))
        p_ij = self.calc_pij(X_dists)
        p_ij = p_ij[~np.eye(N, dtype=bool)].reshape(N, -1)

        Y = np.random.random((N, self.n_components))

        best_i, best_C = -1, np.inf
        self.C_history = []
        for i in tqdm(range(self.n_iter), disable=not self.verbose, desc="iteration"):
            Y_diff = Y[:, np.newaxis, :] - Y
            # remove i=j cases
            Y_diff = Y_diff[~np.eye(N, dtype=bool)].reshape(N, N - 1, -1)
            Y_dists = np.linalg.norm(Y_diff, axis=-1)
            Y_dists = 1 + Y_dists**2
            Y_dists = Y_dists**-1
            # normalize to a probabilty
            q_ij = Y_dists / np.sum(Y_dists)

            C = p_ij * np.log(p_ij / q_ij)
            C = float(np.sum(C))
            self.C_history.append(C)
            if self.verbose:
                print(f"After {i} iterations, {C=}")
            if C < best_C:
                best_i, best_C = i, C
            if best_i + self.n_iter_without_progress < i:
                if self.verbose:
                    print(
                        f"C has not improved for {self.n_iter_without_progress} steps. Stopping early."
                    )
                return Y
            # grad step
            grad_C_yi = 4 * np.sum(
                (p_ij - q_ij)[..., np.newaxis] * Y_diff * Y_dists[..., np.newaxis],
                axis=1,
            )
            Y -= self.learning_rate * grad_C_yi
        if self.verbose:
            print(f"{C=} after {self.n_iter} iterations")
        # Return Y, the 2D representation of the input data
        return Y

    # Part 2: Transformation of New Data Points
    def transform(self, X_original, Y_original, X_new):
        N = len(X_original)
        n_neighbors = min(N, int(self.transform_n_neighbors))
        Y_new = []

        # Added weights to the distances as following: [1, 1/2, 1/3, 1/4.... 1/n]
        # The minimal distance get's the highest weight, and the weight is reduced as the distance increasing
        W_arr = np.arange(1, n_neighbors + 1, dtype=float) ** -1

        # Calculate the distance array for each one of the test values
        dists = cdist(X_new, X_original)

        # Sort the indices of the distances from minimal to maximal while limit just to the n_neighbors distances
        n_nearest_indexes = np.argsort(dists, axis=1)[:, :n_neighbors]

        # Find the average location based on the indices of the distances, while the distances are weighted (Minimal distance -> max weight)
        Y_new = np.average(Y_original[n_nearest_indexes], weights=W_arr, axis=1)

        return np.array(Y_new)

    def calc_pij(self, X_dists):
        N = len(X_dists)
        min_sigma = np.ones((N, 1)) * self.min_sigma
        max_sigma = np.ones((N, 1)) * self.max_sigma
        # precision is min_sigma / 10
        n_iters = int(np.ceil(np.log2(self.max_sigma / (self.min_sigma / 10))))
        for _ in tqdm(range(n_iters), disable=not self.verbose, desc="finding sigma"):
            curr_sigma = (min_sigma + max_sigma) / 2
            p_ij = -(X_dists**2) / (2 * curr_sigma**2)
            # remove i=j cases
            p_ij = p_ij[~np.eye(N, dtype=bool)].reshape(N, -1)
            # numeric stability in exponent
            p_ij -= p_ij.max(axis=1, keepdims=True)
            p_ij = np.exp(p_ij)
            # normalize to a probabilty
            p_ij = p_ij / p_ij.sum(axis=1, keepdims=True)
            distribution_entropy = (-p_ij * np.log2(p_ij)).sum(axis=1, keepdims=True)
            distribution_perplexity = 2**distribution_entropy
            # update sigma
            min_sigma, max_sigma = (
                np.where(
                    distribution_perplexity < self.perplexity, curr_sigma, min_sigma
                ),
                np.where(
                    distribution_perplexity > self.perplexity, curr_sigma, max_sigma
                ),
            )
        # reconstruct diagonal
        p_ij = np.insert(p_ij, np.arange(0, N**2, N), 0).reshape((N, N))
        return (p_ij + p_ij.T) / (2 * N)

# Load data
Please use the cell below to discuss your dataset choice and why it is appropriate (or not) for this algorithm.

We chose to use MNIST dataset to our algorithm.
MNIST is a very popular yet simple dataset, which is used wordlwide for basic/begginer ML tasks, and has many examples specifically for dimensionality reduction outputs (PCA and tSNE). This will allow us to compare ourself. 
It's usage is about encoding a digit in an image (Assuming the digit is shown as pixels just like one takes a photo of a car plate), and each image is labeled by it's digit value

Using T-SNE algorithm, we can find the relationship and the distances between the images with a very clear and visual way, while since we have just 10 digits in MNIST (0-9), we can classify the images into 10 groups and show in 2D dimension the relationship and distance between them

We split the dataset into train (80%) and test (20%) while the dataset contains 10,000 examples in total

Since the image values are described by grayscale pixel codes (0 -> white to 255 -> black), the digits should be represented as black while the background as white, but in fact the digits are in various range of colors between white and black (gray, dark gray. etc). Then, in order to reduce errors since the colors distribution, we want to normalize it so the digit will be represented with one single pixel code.

In [ ]:
# Loading the data into train / test datasets
N_samples = 1000
dataset = pd.read_csv("./Dataset/mnist_dataset.csv")[:N_samples]

# The X axis is the pixels grayscale matrix that represents the digit
X = dataset.loc[:, "1x1":].values

# The y axis is the actual digit that the the pixels matrix represents
y = dataset.loc[:, "label"].values

X_train, X_test, label_train, label_test = train_test_split(X, y, test_size=0.2)

# Normalizing the grayscale codes by minmax scaler.
sc = MinMaxScaler()
X_train_normalized_opt1 = sc.fit_transform(X_train)
X_test_normalized_opt1 = sc.transform(X_test)

# Normalizing the grayscale codes by standard scaler
sc = StandardScaler()
X_train_normalized_opt2 = sc.fit_transform(X_train)
X_test_normalized_opt2 = sc.transform(X_test)

In [ ]:
dataset.head()

In [ ]:
X_train.shape

In [ ]:
X_train_normalized_opt1.shape

In [ ]:
# This is a ploting of sample train data

fig, axes = plt.subplots(
    4,
    10,
    figsize=(10, 4),
    subplot_kw={"xticks": [], "yticks": []},
    gridspec_kw=dict(hspace=0.1, wspace=0.1),
)
for i, ax in enumerate(axes.flat):
    ax.imshow(
        X_train_normalized_opt1[i].reshape(28, 28),
        cmap="binary",
        interpolation="nearest",
        clim=(0, 16),
    )

In [ ]:
# This is a ploting of sample train data

fig, axes = plt.subplots(
    4,
    10,
    figsize=(10, 4),
    subplot_kw={"xticks": [], "yticks": []},
    gridspec_kw=dict(hspace=0.1, wspace=0.1),
)
for i, ax in enumerate(axes.flat):
    ax.imshow(
        X_train_normalized_opt2[i].reshape(28, 28),
        cmap="binary",
        interpolation="nearest",
        clim=(0, 16),
    )

# t-SNE demonstration 
Demonstrate your t-SNE implementation.

Add plots and figures. The code below is just to help you get started, and should not be your final submission.

Please use the cell below to describe your results and tests.

Describe the difference between your implementation and the sklearn implementation. Hint: you can look at the documentation.

As we can see below, we've tested the module with a various combination of Hypher-Parameters. We ran everything on $N=1000$ MNIST examples, enough to see trends and still run fast at $O(N^2)$ time.

The most important hypher-param is the perplexity. We found that a large number of perplexity causes a very dense and unified groups of digits but with some little mistakes since with high perplexity we can also "catch" neighbors that represents another disits. However, with a low number of perplexity the observabilities will not be unified and scattered along the graph
Therefore, according to our research the optimal perplexity number is something about 30-50.

Regarding learning rate, higher learning rate causes the group be closer to each other, and smaller learning rate makes the groups be more far away, since the Y output values are getting more centralized with higher learning rate value. The optimal learning rate is about 100-200. However, it has less affect on the results than the perplexity, and affects mostly the runtime.

Redarding the number of iterations, we've tested with 1000 and 500 iterations and didn't find meaningful changes. This is also expressed in the loss over time graphs, which show that the loss decreases fast early on then slower as it reaches smaller values.
It is important to note that a large number of iterations can cause overfitting (if we didn't implement early stopping), and smaller one can cause underfitting.

Therefore, according to our experiments, we've found that the best hyper-parameters combination is as follows:
* Perplexity of 50 (As previously told, small enough to not catch neigbors from another digits and large enough to make a unified and clear groups)
* Learning rate of 200 (In order to make the groups unified and clear)
* 1000 iterations

In [ ]:
hypher_pharams_options = [
    {"n_components": 2, "n_iter": 1000, "perplexity": 100, "learning_rate": 200},
    {"n_components": 2, "n_iter": 1000, "perplexity": 50, "learning_rate": 200},
    {"n_components": 2, "n_iter": 1000, "perplexity": 30, "learning_rate": 100},
    {"n_components": 2, "n_iter": 500, "perplexity": 30, "learning_rate": 100},
    {"n_components": 2, "n_iter": 1000, "perplexity": 15, "learning_rate": 50},
    {"n_components": 2, "n_iter": 1000, "perplexity": 50, "learning_rate": 200},
    {"n_components": 2, "n_iter": 1000, "perplexity": 5, "learning_rate": 50},
]

best_hypher_pharams_options = [
    {"n_components": 2, "n_iter": 1000, "perplexity": 50, "learning_rate": 200}
]

results_array = []

for hypher_params in hypher_pharams_options:
    n_components = hypher_params["n_components"]
    n_iter = hypher_params["n_iter"]
    perplexity = hypher_params["perplexity"]
    learning_rate = hypher_params["learning_rate"]
    print(
        f"Running with number of components {n_components}, number of iterations {n_iter}, perplexity {perplexity} and lr {learning_rate}"
    )

    # Run your custom t-SNE implementation
    custom_tsne = CustomTSNE(
        n_components=n_components,
        n_iter=n_iter,
        perplexity=perplexity,
        learning_rate=learning_rate,
    )
    custom_Y = custom_tsne.fit_transform(X_train_normalized_opt1)

    # Run sklearn t-SNE
    sk_tsne = TSNE(n_components=n_components, init="random", perplexity=perplexity)
    sk_Y = sk_tsne.fit_transform(X_train_normalized_opt1)

    results_array.append(
        {
            "custom_Y": custom_Y,
            "sk_Y": sk_Y,
            "custom_kl": custom_tsne.C_history,
            "sk_kl": sk_tsne.kl_divergence_,
        }
    )

In [ ]:
for idx, result in enumerate(results_array):
    hyperparams_string = f"{hypher_pharams_options[idx]['n_iter']} iterations, perplexity {hypher_pharams_options[idx]['perplexity']} and lr {hypher_pharams_options[idx]['learning_rate']}"
    # Visualization of the result
    plt.figure()
    plt.scatter(
        result["custom_Y"][:, 0],
        result["custom_Y"][:, 1],
        s=5,
        c=label_train.astype(int),
        cmap="tab10",
    )
    plt.colorbar()
    plt.title(
        f"MNIST Data Embedded into 2D with Custom t-SNE with {hyperparams_string}"
    )
    plt.show()

    plt.figure()
    plt.scatter(
        result["sk_Y"][:, 0],
        result["sk_Y"][:, 1],
        s=5,
        c=label_train.astype(int),
        cmap="tab10",
    )
    plt.colorbar()
    plt.title(
        f"MNIST Data Embedded into 2D with Sklearn t-SNE with {hyperparams_string}"
    )
    plt.show()

    plt.figure()
    plt.plot(
        np.arange(len(result["custom_kl"])),
        result["custom_kl"],
        label="custom KL divergence over time",
    )
    plt.axhline(
        y=result["sk_kl"], label="sk final KL divergence", color="green", linestyle="--"
    )
    plt.xlabel("step")
    plt.ylabel("loss")
    plt.legend()
    plt.title(f"KL divergence function over time with {hyperparams_string}")
    plt.show()

In terms of visual results, both outputs look good - we have clear seperation of different digits.
The quality of the embeddings is also good - the loss decreases along the learning process, and the final KL divergence is always close between the implementations - it depends on the hyperparams of course, but we did not see any major preference twords one implementation over the that note subject.

As noted before, the most important difference between our and sk-learn's implementation is the running time. Ours runs in $O(N^2)$ while sk's runs in $O(N \log{N})$ which is way better especially as $N$ grows larger.

# t-SNE extension - mapping new samples
Demonstrate your t-SNE transformation procedure.

Add plots and figures.

Please use the cell below t describe your suggested approach in detail. Use formal notations where appropriate.
Describe and discuss your results.

We chose to use the transform method as following:
1. For each one of the test set observabilities, we've calculated the distance from it's grayscale vector to any of the grayscale vectors in the train set
2. We find the top n nearest vectors (While n is the perplexity used in the fit_transofrm method) and we build a new matrix of (n X 2) that contains the original Y values of the n nearest vectors from the train set, sorted from 1st nearest to n nearest (Each Y value contains 2 values, the x axis and the y axis in the target 2D graph)
3. The target Y value of the test grayscale vector is the weighted average of the Y values of the n nearest vectors from the train set. While the weight of each value is $\frac{1}{i}, 1 \leq i \leq n$ (The nearest value is weighted by $1$, the 2nd by $\frac{1}{2}$, the 3rd by $\frac{1}{3}$, and the n by $\frac{1}{n}$)
4. We repeat this for each one of the test values, until we build a final Y vector of shape $(N_{test}, 2)$ representing 2D values to be in the graph.

The reason we compare the distances between the grayscale vectors of each one of the test examples to the train examples, is the assumption that each digit represented by a pixel code is not exactly the same as other pixel code vectors of the same digit, but the distance is minimal to grayscale vectors represents the same digit.
Then we assume that most of the n grayscale nearest vectors represents the same digit as the test sample.

Then we want to locate it around the n grayscale nearest vectors but with more weight to the closest vector and less weight as the vector is getting less close.

With that approach we set the Y vectors of the test samples around the group of their digits.

In order to measure the performance of each one of the examples, I've added two dataframe tables for each one of the experiments:
1. The first table compares each test observability actual digit with it's 10 closets train actual digits (As more neighbors have the same digit as the test observability, then the experiment is better)
2. The second tables measures the 10 closest train distances to each one of the test observabilities 

In [ ]:
for idx, result in enumerate(results_array):
    # Transform new data
    custom_Y_new = custom_tsne.transform(
        X_train_normalized_opt1, result["custom_Y"], X_test
    )

    # Visualization of the result
    plt.figure()
    plt.scatter(
        result["custom_Y"][:, 0],
        result["custom_Y"][:, 1],
        s=5,
        c=label_train.astype(int),
        cmap="tab10",
    )
    plt.scatter(
        custom_Y_new[:, 0],
        custom_Y_new[:, 1],
        marker="*",
        s=50,
        linewidths=0.5,
        edgecolors="k",
        c=label_test.astype(int),
        cmap="tab10",
    )
    plt.colorbar()
    plt.title(
        f"MNIST Data Embedded into 2D with Custom t-SNE with {hypher_pharams_options[idx]['n_iter']} iterations, perplexity {hypher_pharams_options[idx]['perplexity']} and lr {hypher_pharams_options[idx]['learning_rate']}"
    )
    plt.show()

    # Top 10 nearest neighbor of each test set
    dists = cdist(custom_Y_new, result["custom_Y"])
    n_nearest_digits = label_train[np.argsort(dists, axis=1)[:, :10]]
    n_nearest_distances = np.sort(dists, axis=1)[:, :10]

    df1 = pd.DataFrame(label_test, columns=["actual digit"])
    df2 = pd.DataFrame(
        n_nearest_digits,
        columns=[f"{idx+1}th closest digit" for idx in range(n_nearest_digits.shape[1])],
    )
    df3 = pd.DataFrame(
        n_nearest_distances,
        columns=[
            f"{idx+1}th closest distance" for idx in range(n_nearest_distances.shape[1])
        ],
    )
    df_digits = df1.join(df2)
    df_distances = df1.join(df3)

    most_common_neigbors_digits = np.array(
        [np.bincount(row).argmax() for row in n_nearest_digits]
    )
    digits_matrix = confusion_matrix(label_test, most_common_neigbors_digits)
    cm_display = ConfusionMatrixDisplay(confusion_matrix=digits_matrix)
    cm_display.plot(cmap="magma")
    plt.title(
        "Actual test digit vs the most common digit in the nearest 10 train neighbors"
    )
    plt.grid(False)
    plt.show()

    df_measures = pd.DataFrame(
        [
            [
                precision_score(
                    label_test, most_common_neigbors_digits, average="weighted"
                ),
                recall_score(
                    label_test, most_common_neigbors_digits, average="weighted"
                ),
            ]
        ],
        columns=["precision", "recall"],
    )
    display(df_measures)

    print("10 Closests train actual digits for each test digit")
    display(df_digits[:20])
    print("10 Closests train distances for each test oservability")
    display(df_distances[:20])

# Use of generative AI
Please use the cell below to describe your use of generative AI in this assignment. 

We used ChatGPT to help us utilize numpy vectorised functions more effieintly. It also proposed subtracting the maximum when calculating exponents, in order to avoid very large/small values.